<a href="https://colab.research.google.com/github/gmb408/OSSP-KBO-prediction/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# 데이터 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
import matplotlib.pyplot as plt 
import numpy as np 
from scipy.stats import norm 

In [38]:
result = pd.read_csv("/content/statiz_pre.csv", index_col=0)

In [39]:
# 이상치 결측치 제거
result.drop(result.loc[result['타석']<100].index, inplace=True)
result=result.reset_index(drop=True)

In [40]:
# 단타 feature 생성
result['1타'] = result['안타'] - result['2타'] - result['3타'] - result['홈런']
# 순출루율
result['isoD'] = result['출루'] - result['타율']
# 순창타율
result['isoP'] = result['장타'] - result['타율']

In [41]:
# 사용 features 명시
infos = ['ID','이름','생일','팀','시즌','포지션','나이']
stats = ['G','타수','타석','득점','안타','1타','2타','3타','홈런','루타','타점','볼넷','삼진'\
        ,'타율','출루','isoD','장타','isoP','OPS','wOBA','wRC+','WAR+']
result = result[infos+stats]

In [42]:
# 해당 시즌 이전까지의 누적 기록
def get_ative(player_id, season, stat=stats ,df=result):
    try:
        result_cum = df[(df['ID']==player_id) & (df['시즌'] <= season)][stat]
        result_cum = len(result_cum)
        
    except:
        result_cum['연차'] = np.nan 
        result_cum = pd.Series(result_cum)
    
    return result_cum


In [43]:
# 해당 시즌 이전까지의 누적 기록
def get_cumulative(player_id, season, stat=stats ,df=result):
    try:
        result_cum = df[(df['ID']==player_id) & (df['시즌'] < season)][stat]
        result_cum = result_cum.mean()
        
    except:
        result_cum = [np.nan for i in stat]
        result_cum = pd.Series(result_cum)

    
    return result_cum

In [44]:
# 모든 stats에 대한 누적 stat 나이별 스텟
stats_cum = [i + '_누적' for i in stats]

In [45]:
# 누적 stat 데이터프레임에 반영
result[stats_cum] = result.apply(lambda x: get_cumulative(x['ID'], x['시즌']), axis=1)

In [46]:
# lag stat (직전 몇년의 기록)
def get_lag(player_id, season, lag, stat=stats, df=result):
    try:
        result_lag = df[(df['ID']==player_id)&(df['시즌'] == season-lag)][stat].values[0]
        result_lag = pd.Series(result_lag)
        
    except:
        result_lag = [np.nan for i in stat]
        result_lag = pd.Series(result_lag)
        
    return result_lag


In [47]:
# 직전 1,2,3년의 기록
stats_lag1 = [i + '_LAG1' for i in stats]
stats_lag2 = [i + '_LAG2' for i in stats]
stats_lag3 = [i + '_LAG3' for i in stats]

In [48]:
# lag1
result[stats_lag1] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=1), axis=1)

In [49]:
# lag2
result[stats_lag2] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=2), axis=1)

In [50]:
# lag3
result[stats_lag3] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=3), axis=1)

In [51]:
# 내년 기록 (OPS, 장타, 출루)
# 데이터 프레임의 정답 셋 만들기 위함

def get_next_year(player_id, season, stat=['장타','출루','OPS'],df=result):
    try:
        result = df[(df['ID']==player_id)&(df['시즌']==season+1)][stat].values[0]
        result = pd.Series(result)
    except:
        result = [np.nan for i in stat]
        result = pd.Series(result)
        
    return result

In [52]:
# 장타, 출루, OPS의 내년 기록
stats_next = [i + '_NEXT' for i in ['장타', '출루', 'OPS']]
result[stats_next] = result.apply(lambda x: get_next_year(x['ID'], x['시즌']), axis=1)

In [53]:
result['연차'] = result.apply(lambda x: get_ative(x['ID'], x['시즌']), axis=1)

In [54]:
# 변환 데이터 저장
result.to_csv("/content/statiz_Final.csv")
result.columns

Index(['ID', '이름', '생일', '팀', '시즌', '포지션', '나이', 'G', '타수', '타석',
       ...
       '장타_LAG3', 'isoP_LAG3', 'OPS_LAG3', 'wOBA_LAG3', 'wRC+_LAG3',
       'WAR+_LAG3', '장타_NEXT', '출루_NEXT', 'OPS_NEXT', '연차'],
      dtype='object', length=121)